In [165]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics
import json
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import statsmodels.api as sm
from scipy import stats
from statsmodels.stats.outliers_influence import variance_inflation_factor

df = pd.read_csv(r"C:\Users\benja\OneDrive\Documents\Classes\Machine Learning\Project 2\daniel_test.csv")
df = df.drop(columns = [ 'country', 'year'], axis = 1)
df_filled = df.dropna()
# Define the dependent variable and column names
dependent_variable = 'gini_disp_ext'
column_names = df.columns.tolist()
# Remove the dependent variable from the list of column names
column_names.remove(dependent_variable)
important_columns = ['latent_modernization', 'latent_climate', 'latent_earlydev', 'euro2007',  'Populationdensitypeoplepers_ln', 'gini_disp_ext']
X = ['latent_modernization', 'latent_climate', 'latent_earlydev', 'euro2007',  'Populationdensitypeoplepers_ln']
Y = dependent_variable

#get summary of data and save it to a csv file
df[important_columns].describe().to_csv('SummaryRaw.csv', index = False)
df_filled[important_columns].describe().to_csv('SummaryFiltered.csv', index= False)

#print(df)
#print(df_filled)

# Get the point-biserial correlation of the dummy variables compared to the gini
#correlation_coefficient, p_value = stats.pointbiserialr(df_filled['legor_so'], df_filled['gini_disp_ext'])
#print(f"Point-Biserial Correlation Coefficient: {correlation_coefficient}")
#print(f"P-Value: {p_value}")

#Calculate the correlation matrix for all the independent variables, then narrow it down to X independent variables
correlation_matrix = df.corr()
correlation_X = df[important_columns].corr()
#print("Correlation Matrix:")
#print(correlation_X)

#Calculate the covariance matrix, then narrowed down to X independent variables
numpy = df_filled.to_numpy()
numpy_X = df_filled[important_columns].to_numpy()
covariance_matrix = np.cov(numpy_X, rowvar=False)
#print("Covariance Matrix:")
#print(covariance_matrix)


#Calculate the Variance Inflation Matrix(VIF)
important_dfilled = df_filled[X]
variables = important_dfilled.values
vif_values = [variance_inflation_factor(variables, i) for i in range(important_dfilled.shape[1])]
# Create a DataFrame to display the results
vif_df = pd.DataFrame({'Variable': important_dfilled.columns, 'VIF': vif_values})
print(vif_df)

# Create scatter plots for each column with respect to the dependent variable
# for column in column_names:
#    plt.figure(figsize=(8, 6))
#    plt.scatter(df[column], df[dependent_variable], alpha=0.5)
#    plt.title(f'Scatter Plot of {column} vs. {dependent_variable}')
#    plt.xlabel(column)
#    plt.ylabel(dependent_variable)
#    plt.grid(True)
#     plt.savefig(f'plot_{column}.png', dpi=300)
#     plt.show()

#START OF REGRESSION CODE
final_df = pd.read_csv(r"C:\Users\benja\OneDrive\Documents\Classes\Machine Learning\Project 2\Final Filtered Data.csv")
x = final_df.loc[:, X].values
y = final_df.loc[:, Y].values
#normalize x
x_norm = StandardScaler().fit_transform(x)

#Create LINEAR REGRESSION MODEL FROM OLS
#x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
# Get the coefficients and variable names
coefficientsOLS = model.params  # Include the intercept
variable_names = X
equation = " + ".join(f"{b:.4f}*{var}" for b, var in zip(coefficientsOLS, variable_names))
print(model.summary())
print(f"Regression Equation: {equation}")

#CREATE REGRESSION MODEL FROM SCIKIT LINEAR REGRESSION
#Assuming X is your feature matrix (independent variables) and y is your target variable (dependent variable)
model = LinearRegression()
model.fit(x, y)

# To get the coefficients (slopes) and intercept
coefficients = model.coef_
intercept = model.intercept_

# To make predictions
predictions = model.predict(x)
print(coefficients)
print(intercept)
y_pred = model.predict(x)
r_squared = r2_score(y, y_pred)
print(f"R-squared: {r_squared}")

#POLYNOMIAL REGRESSION
#creates the training set and test set; test_size = 0.2 means 80% training 20% test; random_state is just an arbitrary number 
# that uses returns the same random values every time 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)
poly = PolynomialFeatures(degree=1)
X_train_poly = poly.fit_transform(x_train)
X_test_poly = poly.transform(x_test)

# Fit a linear regression model on the polynomial features
model = LinearRegression()
model.fit(X_train_poly, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test_poly)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r_squared = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r_squared}")

                         Variable       VIF
0            latent_modernization  1.839760
1                  latent_climate  2.452387
2                 latent_earlydev  1.294030
3                        euro2007  3.389650
4  Populationdensitypeoplepers_ln  1.830087
                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.905
Model:                            OLS   Adj. R-squared (uncentered):              0.903
Method:                 Least Squares   F-statistic:                              366.0
Date:                Tue, 10 Oct 2023   Prob (F-statistic):                    4.31e-96
Time:                        19:44:14   Log-Likelihood:                         -776.31
No. Observations:                 197   AIC:                                      1563.
Df Residuals:                     192   BIC:                                      1579.
Df Model:                       